In [ ]:
import quaternion
import numpy as np
import scipy.optimize as opt
import plotly.graph_objects as go

from scipy.integrate import solve_ivp
from tqdm import tqdm

In [ ]:
r_moon  = 1737.1e3
mu_moon = 4.9048695e12

In [ ]:
def gravity(r, mu):
    '''
    Calculates the gravitational acceleration at a point
    :param r: position vector
    :param mu: gravitational parameter of the central body
    :return: gravitational acceleration vector
    '''
    return -mu*r/(np.linalg.norm(r)**3)

In [ ]:
def lla_to_cart(lla_pos, R):
    lat = np.radians(lla_pos[0])
    lon = np.radians(lla_pos[1])
    alt = lla_pos[2]

    x = R * np.cos(lat) * np.cos(lon)
    y = R * np.cos(lat) * np.sin(lon)
    z = R * np.sin(lat) + alt
    return np.array([x, y, z])

def cart_to_lla(cart_pos, R):
    x = cart_pos[0]
    y = cart_pos[1]
    z = cart_pos[2]
    
    lat = np.arcsin(z/R)
    lon = np.arctan2(y, x)
    alt = np.linalg.norm(cart_pos) - R
    return np.array([np.degrees(lat), np.degrees(lon), alt])

def mcmf2mci(pcpf_state, days_since_j2000):
    q_pcpf_to_inertial, omega_pcpf = get_mcmf_to_mci(days_since_j2000)
    pci_state       = np.zeros(6)
    pci_state[0:3]  = quaternion.rotate_vectors(q_pcpf_to_inertial, pcpf_state[0:3])
    pci_state[3:6]  = quaternion.rotate_vectors(q_pcpf_to_inertial, pcpf_state[3:6]) \
                        + np.cross(omega_pcpf, pci_state[0:3])
    return pci_state

def get_mcmf_to_mci(days_since_j2000):
    degrees_per_day     = 360 / 27.321661
    rot_offset          = 0
    gamma               = np.radians(degrees_per_day*days_since_j2000 + rot_offset)
    q_pcpf_to_inertial  = quaternion.from_rotation_vector(gamma*np.array([0, 0, 1]))
    omega_pcpf  = np.array([0, 0, np.radians(degrees_per_day/(24*3600))])
    return q_pcpf_to_inertial, omega_pcpf

def get_inertial_to_enu(lla_pos):
    lat = np.radians(lla_pos[0])
    lon = np.radians(lla_pos[1])
    R_mcmf_to_enu = np.array([[-np.sin(lon),              np.cos(lon),                0],
                              [-np.sin(lat)*np.cos(lon),  -np.sin(lat)*np.sin(lon),   np.cos(lat)],
                              [np.cos(lat)*np.cos(lon),   np.cos(lat)*np.sin(lon),    np.sin(lat)]])
    return quaternion.from_rotation_matrix(R_mcmf_to_enu)

def convert_inertial_state_to_enu(pos, lla_ref):
    pos_ref = lla_to_cart(lla_ref, r_moon)
    q_inertial_to_enu = get_inertial_to_enu(lla_ref)
    pos_enu = quaternion.rotate_vectors(q_inertial_to_enu, pos-pos_ref)
    return pos_enu

def get_inertial_to_lvlh(state):
    '''
    Calculates the rotation matrix from the LVLH frame to the inertial frame
    :param state: current state vector [x, y, z, vx, vy, vz]
    :return: rotation matrix from the LVLH frame to the inertial frame
    '''
    r = state[0:3]
    v = state[3:6]
    h = np.cross(r, v)

    r_normalized    = unit(r)
    h_normalized    = unit(h)
 
    # Rotation matrix from Hill to Inertial
    R_lvlh_to_inertial = np.array([np.cross(-h_normalized, -r_normalized), -h_normalized, -r_normalized]).T
    q_lvlh_to_inertial = quaternion.from_rotation_matrix(R_lvlh_to_inertial)
    q_inertial_to_lvlh = q_lvlh_to_inertial.conjugate()

    return q_inertial_to_lvlh

# Get inertial to Target Coordinate System as described in 
# R-567 GUIDANCE SYSTEM OPERATIONS PLAN FOR MANNED LM EARTH ORBITAL AND
# LUNAR MISSIONS USING PROGRAM LUMINARY IE SECTIONS GUIDANCE EQUATIONS(Rev. 11)
def get_inertial_to_TCS(init_state, target_state):
    # initial position vector of vehicle
    r_0 = init_state[0:3]

    # state vector of target
    r_c = target_state[0:3]
    v_c = target_state[3:6]

    q = np.cross(v_c, r_c)
    Q = unit(q)

    s = np.cross(r_0, Q)
    S = unit(s)

    P = np.cross(Q, S)

    R = np.array([Q, S, P])
    q_inertial_to_TCS = quaternion.from_rotation_matrix(R)
    return q_inertial_to_TCS

def get_inertial_to_LVCS(state, target_state):
    r = state[0:3]

    r_c = target_state[0:3]
    v_c = target_state[3:6]

    u_R = unit(r)

    q = np.cross(v_c, r_c)
    Q = unit(q)

    u_z = np.cross(u_R, Q)
    u_Z = unit(u_z)

    u_Y = np.cross(u_Z, u_R)

    R = np.array([u_R, u_Y, u_Z])
    q_inertial_to_LVCS = quaternion.from_rotation_matrix(R)
    return q_inertial_to_LVCS

def unit(v):
    return v/np.linalg.norm(v)

In [ ]:
def E_func(E, M, e):
        return E - e*np.sin(E) - M
def E_prime(E, M, e):
    return 1 - e*np.cos(E)

def eccentric_anomaly(M, e):
    E = opt.newton(E_func, M+e, fprime=E_prime, args=(M,e),tol=1e-10,)
    return E

def mean_to_true_anomaly(M, e):
    E = eccentric_anomaly(M, e)
    f = 2*np.arctan2(np.tan(E/2)*np.sqrt(1+e), np.sqrt(1-e))
    if f > np.pi:
        f -= 2*np.pi
    if f < -np.pi:
        f += 2*np.pi
    return f
def disp_state_vec(state):
    print(f"R:\t [{state[0]}, {state[1]}, {state[2]}] m")
    print(f"V:\t [{state[3]}, {state[4]}, {state[5]}] m/s\n")

def disp_kep_elem(kep):
    print(f"Semimajor Axis (a):\t\t {kep[0]} m")
    print(f"Eccentricity (e):\t\t {kep[1]}")
    print(f"Inclination (i):\t\t {np.degrees(kep[2])}º")
    print(f"RAAN (Ω):\t\t\t {np.degrees(kep[3])}º")
    print(f"Argument of Periapse (𝜔):\t {np.degrees(kep[4])}º")
    print(f"True Anomaly (f):\t\t {np.degrees(kep[5])}º\n")

def state_vec_to_keplerian(state, mu):
    a = semimajor_axis(state, mu)
    e = np.linalg.norm(eccentricity(state, mu))
    i = inclination(state)
    raan = RAAN(state)
    omega = argument_of_periapse(state, mu)
    f = true_anomaly(state, mu)
    return np.array([a, e, i, raan, omega, f])

def keplerian_to_state_vec(kep, mu):
    pos = position(kep, mu)
    vel = velocity(kep, mu)

    state_vec = np.array([pos[0], pos[1], pos[2], vel[0], vel[1], vel[2]])
    input = kepDeg2Rad(kep)
    check = state_vec_to_keplerian(state_vec, mu)
    if np.allclose(input, check, rtol=1e-04, atol=1e-05):
        return state_vec
    else:
        print("WARNING: Keplerian elements do not match")
        disp_kep_elem(input)
        disp_kep_elem(check)
        return state_vec

def semimajor_axis(state, mu):
    r = np.linalg.norm(state[0:3])
    v = np.linalg.norm(state[3:6])
    if r == 0:
        return 0
    a = 1/(2/r - v**2/mu)
    return a

def eccentricity(state, mu):
    r = state[0:3]
    v = state[3:6]
    r_mag = np.sqrt(np.dot(r, r))
    v_mag = np.sqrt(np.dot(v, v))
    if r_mag == 0:
        return 0
    e = (v_mag**2/mu - 1/r_mag)*r - 1/mu*np.dot(r, v)*v
    return e

def inclination(state):
    r = state[0:3]
    v = state[3:6]
    h = np.cross(r, v)
    h_mag = np.sqrt(np.dot(h, h))
    if h_mag == 0:
        return 0
    incl = np.arccos(np.dot(h/h_mag, np.array([0, 0, 1])))
    return incl

def node_vector(state):
    r = state[0:3]
    v = state[3:6]
    h = np.cross(r, v)
    n = np.cross(np.array([0, 0, 1]), h)
    return n

def RAAN(state):
    node = node_vector(state)
    node_mag = np.sqrt(np.dot(node, node))
    if node_mag == 0:
        return 0
    omega = np.arccos(np.dot(node, np.array([1, 0, 0])) / node_mag)
    if np.dot(node, np.array([0, 1, 0])) < 0:
        omega = 2*np.pi - omega
    return omega

def argument_of_periapse(state, mu):
    node = node_vector(state)
    node_mag = np.sqrt(np.dot(node, node))
    e = eccentricity(state, mu)
    e_mag = np.sqrt(np.dot(e, e))
    if node_mag == 0 or e_mag == 0:
        return 0
    omega = np.arccos(np.dot(node, e)/(node_mag*e_mag))
    if (np.dot(e, np.array([0, 0, 1])) < 0):
        omega = 2*np.pi - omega
    return omega

def true_anomaly(state, mu):
    r = state[0:3]
    e = eccentricity(state, mu)
    r_mag = np.sqrt(np.dot(r, r))
    e_mag = np.sqrt(np.dot(e, e))
    if r_mag == 0 or e_mag == 0:
        return 0
    if np.isclose(np.dot(e, r)/(e_mag*r_mag), 1):
        return 0
    f = np.arccos(np.dot(e, r)/(e_mag*r_mag))
    while f >= 2*np.pi:
        f -= 2*np.pi
    while f <= -2*np.pi:
        f += 2*np.pi
    return f

def angular_momentum_from_OE(a, e, mu):
    if e >=0 and e < 1:
        return np.sqrt(mu*a*(1-e**2))
    elif e < 0:
        return np.sqrt(mu*a*(e**2-1))


def position(kep, mu):
    a = kep[0]
    e = kep[1]
    i = np.radians(kep[2])
    raan = np.radians(kep[3])
    omega = np.radians(kep[4])
    f = np.radians(kep[5])

    r = a*(1-e**2)/(1+e*np.cos(f))
    theta = omega + f
    pos = r * np.array([np.cos(theta)*np.cos(raan) - np.cos(i)*np.sin(raan)*np.sin(theta),
                        np.cos(theta)*np.sin(raan) + np.cos(i)*np.cos(raan)*np.sin(theta),
                        np.sin(i)*np.sin(theta)])
    return pos

def velocity(kep, mu):
    a = kep[0]
    e = kep[1]
    inc = np.radians(kep[2])
    raan = np.radians(kep[3])
    omega = np.radians(kep[4])
    f = np.radians(kep[5])

    theta = omega + f

    h = angular_momentum_from_OE(a, e, mu)
    theta = omega + f
    vel_vec = mu/h*np.array([-(np.cos(raan)*(np.sin(theta) + e*np.sin(omega)) + np.sin(raan)*(np.cos(theta)+ e*np.cos(omega))*np.cos(inc)),
                             -(np.sin(raan)*(np.sin(theta) + e*np.sin(omega)) - np.cos(raan)*(np.cos(theta) + e*np.cos(omega))*np.cos(inc)),
                             (np.cos(theta) + e*np.cos(omega))*np.sin(inc)])              
    return vel_vec

def kepRad2Deg(kep):
    return np.array([kep[0], kep[1], np.degrees(kep[2]), np.degrees(kep[3]), np.degrees(kep[4]), np.degrees(kep[5])])

def kepDeg2Rad(kep):
    return np.array([kep[0], kep[1], np.radians(kep[2]), np.radians(kep[3]), np.radians(kep[4]), np.radians(kep[5])])   

In [ ]:
def get_torque_cmd(q_inertial_to_body, omega, q_inertial_to_target):
    k_p = 500000
    k_d = 80000
    q_i2b   = quaternion.from_float_array(q_inertial_to_body)
    q_i2t   = quaternion.from_float_array(q_inertial_to_target)
    omega   = omega

    dq      = get_error_quat(q_i2b, q_i2t)
    dq_vec  = np.array([dq.x, dq.y, dq.z])

    torque_cmd = -k_p * dq_vec - k_d * omega
    return torque_cmd

def get_error_quat(q1, q2):
    if np.dot(quaternion.as_float_array(q1), quaternion.as_float_array(q2)) < 0:
        q1 = -q1
    return q1 * q2.inverse()

def get_error_quats(q1s, q2s):
    q_errs = np.zeros((len(q1s),4))
    for i in range(len(q1s)):
        q1          = quaternion.from_float_array(q1s[i])
        q2          = quaternion.from_float_array(q2s[i])
        dq          = get_error_quat(q1, q2)
        q_err       = quaternion.as_float_array(dq)
        q_errs[i]   = q_err
    return q_errs

class P_12():

    def __init__(self, time, state_init, guid_params, dt):
        self.state_init                 = state_init
        self.q_inertial_to_body_init    = quaternion.from_float_array(state_init[6:10])
        self.curr_mode                  = -1
        self.guid_params                = guid_params
        self.target_veh_state           = guid_params[0]
        self.mode_switch_states         = guid_params[1]

        guid_auto_input_prms            = guid_params[3]
        veh_params                      = guid_auto_input_prms[0]
        self.T_APS                      = veh_params[3]
        self.Isp_APS                    = veh_params[4]
        self.V_e                        = self.Isp_APS*9.81
        self.m_dot                      = self.T_APS/self.V_e

        self.t_go                       = guid_auto_input_prms[1]
        self.t_3                        = guid_auto_input_prms[2]
        self.t_2                        = guid_auto_input_prms[3]

        guid_man_input_params           = guid_params[2]
        self.t_ig                       = guid_man_input_params[0]
        self.R_D                        = guid_man_input_params[1]
        self.Y_D                        = guid_man_input_params[2]
        self.v_CO_LVCS                  = np.array(guid_man_input_params[3:6])

        self.dt                        = dt

        self.t_meco                    = 4
        self.meco_timer_init           = False

        self.t_calc                     = time
        self.calculate_guid_coeffs(self.t_calc, state_init)

    def calc_g_eff(self, pos, vel):
        H                       = np.linalg.norm(np.cross(pos, vel))
        g_eff                   = H**2/(np.linalg.norm(pos))**3 - np.linalg.norm(gravity(pos, mu_moon))
        return g_eff 

    def calculate_guid_coeffs(self, time, state):
        pos                     = state[0:3]
        vel                     = state[3:6]
        mass                    = state[13]
        self.tau                = mass / self.m_dot
        q_inertial_to_LVCS      = get_inertial_to_LVCS(state, self.target_veh_state)
        cur_vel_LVCS            = quaternion.rotate_vectors(q_inertial_to_LVCS, vel)
        g_eff                   = self.calc_g_eff(pos, vel)
        self.rel_vel_LVCS       = self.v_CO_LVCS - cur_vel_LVCS
        V_g                     = np.linalg.norm(self.rel_vel_LVCS - 1/2*self.t_go*g_eff*np.array([1,0,0]))
        
        self.t_go               = self.tau*V_g/self.V_e * (1 - 1/2 * V_g/self.V_e)
        
        R           = np.linalg.norm(state[0:3])
        u_R         = unit(pos)
        q           = np.cross(self.target_veh_state[3:6], self.target_veh_state[0:3])
        Q           = unit(q)
        Y           = self.R_D*np.arcsin(np.dot(u_R, Q))   

        if self.t_go >= self.t_2:
            self.L      = np.log(1 - self.t_go/self.tau)
            self.D_12   = self.tau + self.t_go/self.L
            if self.t_go > self.t_3:
                self.D_21   = self.t_go - self.D_12
                self.E      = self.t_go/2 - self.D_21 

                self.B      = (self.D_21*self.rel_vel_LVCS[0] - (self.R_D - R - cur_vel_LVCS[0]*self.t_go))/(self.t_go*self.E)
                self.D      = (self.D_21*self.rel_vel_LVCS[1] - (self.Y_D - Y - cur_vel_LVCS[1]*self.t_go))/(self.t_go*self.E)

                self.B = np.clip(self.B, -0.3048*self.tau, 0)

                self.t_calc = time
            else:
                self.B  = 0
                self.D  = 0
            self.A  = -self.D_12*self.B - self.rel_vel_LVCS[0] / self.L
            self.C  = -self.D_12*self.D - self.rel_vel_LVCS[1] / self.L


    def set_guidance_mode(self, time, state):
        current_mode        = self.curr_mode

        q_inertial_to_LVCS  = get_inertial_to_LVCS(state, self.target_veh_state)
        radial_rate         = quaternion.rotate_vectors(q_inertial_to_LVCS, state[3:6])[0]
        if current_mode == -1 and time > self.mode_switch_states[0]:
            self.curr_mode = 0
            self.init_state = state
            self.calculate_guid_coeffs(time, state)
        elif current_mode == 0 and time > self.mode_switch_states[0] + self.mode_switch_states[1]:
            self.curr_mode = 1
        elif current_mode == 1 and radial_rate > self.mode_switch_states[2]:
            self.curr_mode = 2
        elif current_mode == 2 and self.t_meco <= 0:
            self.curr_mode = 3
    
    def get_sc_commands(self, t, state):
        q_inertial_to_body  = quaternion.from_float_array(state[6:10])
        pos                 = state[0:3]
        vel                 = state[3:6]
        mass                = state[13]
        
        if self.curr_mode == -1: #Pre-launch
            burn                    = False
            accel_dir              = np.zeros(3)
            q_inertial_to_target    = self.q_inertial_to_body_init
        elif self.curr_mode == 0: #Lift-off and Attitude Hold
            burn                    = True
            accel_dir               = np.array([1, 0, 0])
            q_inertial_to_target    = self.q_inertial_to_body_init
        elif self.curr_mode == 1: #Azimuth Align
            burn                    = True
            accel_dir               = np.array([1, 0, 0])
            q_inertial_to_target    = get_inertial_to_LVCS(state, self.target_veh_state)
        elif self.curr_mode == 2: #Orbit Insert
            a_T                     = self.T_APS/mass
            g_eff                   = self.calc_g_eff(pos, vel)

            if t - self.t_calc > 2:
                self.calculate_guid_coeffs(t, state)

            a_TR    = 1/self.tau * (self.A + self.B*(t - self.t_calc)) - g_eff
            a_TY    = 1/self.tau * (self.C + self.D*(t - self.t_calc))

            a_H = np.array([a_TR, a_TY])

            if np.linalg.norm(a_H) >= a_T:
                a_TZ = 0
                a_H_command = a_T* unit(a_H)
            else:
                a_TZ = np.sqrt(a_T**2 - np.linalg.norm(a_H)**2) * np.sign(self.rel_vel_LVCS[2])
                a_H_command = a_H
                
            accel_LVCS  = np.hstack((a_H_command, a_TZ))

            accel_dir   = unit(accel_LVCS)
            q_inertial_to_target = get_inertial_to_lvlh(state)
            burn = True

            if self.t_go <=4 and not self.meco_timer_init:
                self.t_meco          = 4
                self.meco_timer_init = True

            if self.meco_timer_init:
                self.t_meco -= self.dt

        elif self.curr_mode == 3: #Coasting
            burn                    = False
            accel_dir               = np.zeros(3)
            q_inertial_to_target    = get_inertial_to_lvlh(state)

        return burn, accel_dir, quaternion.as_float_array(q_inertial_to_target)

In [ ]:
def state_dot(t, state, veh_params, sc_commands, guidance):
    pos                 = state[0:3]
    vel                 = state[3:6]
    q_inertial_to_body  = quaternion.from_float_array(state[6:10])
    omega               = state[10:13]
    mass                = state[13]

    mu      = veh_params[0]
    J       = veh_params[1]
    m_dry   = veh_params[2]
    T_APS   = veh_params[3]
    Isp_APS = veh_params[4]
    grav    = gravity(pos, mu)

    burn_APS                = sc_commands[0]
    a_dir_LVCS              = sc_commands[1]
    q_inertial_to_target    = sc_commands[2]

    if burn_APS and mass > m_dry:
        T_mag   = T_APS
    else: 
        T_mag   = 0
    
    T_inertial  = quaternion.rotate_vectors(get_inertial_to_LVCS(state, guidance.target_veh_state).conjugate(), T_mag*a_dir_LVCS)
    L_body      = get_torque_cmd(state[6:10], omega, q_inertial_to_target)

    p_dot   = vel
    v_dot   = grav + T_inertial/mass
    q_dot   = 1/2 * np.quaternion(0, omega[0], omega[1], omega[2]) * q_inertial_to_body
    alpha   = np.linalg.inv(J) @ (L_body - np.cross(omega,  J @ omega))
    m_dot   = -T_mag / (Isp_APS * 9.81)
    x_dot   = np.hstack((p_dot, v_dot, quaternion.as_float_array(q_dot), alpha, m_dot))
    return x_dot

In [ ]:
def simulate(init_state, times, static_params, guidance_params):
    state = init_state
    dt = times[1] - times[0]

    state_hist                  = np.zeros((len(times), 14))
    q_inertial_to_target_hist   = np.zeros((len(times), 4))
    enu_state_hist              = np.zeros((len(times), 6))
    LVCS_state_hist             = np.zeros((len(times), 5))
    thrust_dir_hist             = np.zeros((len(times), 3))

    guidance = P_12(times[0], init_state, guidance_params, dt)

    state_hist[0,:]                 = state
    q_inertial_to_target_hist[0]    = state[6:10]

    q_inertial_to_LVCS = get_inertial_to_LVCS(state, guidance.target_veh_state)
    R = np.linalg.norm(state[0:3])
    u_R = unit(state[0:3])
    q = np.cross(guidance.target_veh_state[3:6], guidance.target_veh_state[0:3])
    Q = unit(q)
    Y = guidance.R_D*np.arcsin(np.dot(u_R, Q))
    vel_LVCS = quaternion.rotate_vectors(q_inertial_to_LVCS, state[3:6])
    LVCS_state_hist[0] = np.hstack((R, Y, vel_LVCS))
    
    enu_state_hist[0, 0:3]  = convert_inertial_state_to_enu(state[0:3], cart_to_lla(state[0:3], r_moon))
    enu_state_hist[0, 3:6]  = state[3:6]

    idx_events              = []
    curr_mode = guidance.curr_mode
    for i in tqdm(range(0, len(times)-1)):
        t_now = times[i]
        guidance.set_guidance_mode(t_now, state)
        if guidance.curr_mode != curr_mode:
            idx_events.append(i)

        curr_mode   = guidance.curr_mode
        if np.linalg.norm(state[0:3]) > r_moon and curr_mode > -1:
            sc_commands = guidance.get_sc_commands(t_now, state)
            sol         = solve_ivp(state_dot, t_span=[t_now, t_now+dt],y0=state, 
                                    args=(static_params,sc_commands,guidance), method='RK45', max_step=0.01)

            state = sol.y[:,-1]
        state_hist[i+1]                 = state
        if guidance.curr_mode == -1:
            q_inertial_to_target_hist[i+1]  = state[6:10]
            thrust_dir_hist[i]              = np.zeros(3)
        else:
            q_inertial_to_target_hist[i+1]  = sc_commands[2]
            thrust_dir_hist[i]              = sc_commands[1]
        enu_state_hist[i+1, 0:3]        = convert_inertial_state_to_enu(state[0:3], cart_to_lla(state_hist[0,0:3], r_moon))
        enu_state_hist[i+1, 3:6]        = state[3:6]

        q_inertial_to_LVCS = get_inertial_to_LVCS(state, guidance.target_veh_state)
        R = np.linalg.norm(state[0:3])
        u_R = unit(state[0:3])
        q = np.cross(guidance.target_veh_state[3:6], guidance.target_veh_state[0:3])
        Q = unit(q)
        Y = guidance.R_D*np.arcsin(np.dot(u_R, Q))
        vel_LVCS = quaternion.rotate_vectors(q_inertial_to_LVCS, state[3:6])
        LVCS_state_hist[i+1] = np.hstack((R, Y, vel_LVCS))
    return [times, state_hist, enu_state_hist, q_inertial_to_target_hist, LVCS_state_hist, thrust_dir_hist, idx_events]

In [ ]:
save_file = "/Users/parth/Documents/GitHub/Grad Coursework/AA528/FinalProject/sim.npy"

launch_site = np.array([0.67409, 23.47298, 1])
mcmf_pos    = lla_to_cart(launch_site, r_moon)
mcmf_state  = np.hstack((mcmf_pos, np.zeros(3)))
mci_state   = mcmf2mci(mcmf_state, 0)

csm_semimajor   = (101.00e3 + 122.00e3)/2 + r_moon
csm_e           = 0.00568
csm_inc         = 0.00
csm_raan        = 0.00
csm_omega       = 0.00
csm_f           = 0.00

csm_orbital_elements    = np.array([csm_semimajor, csm_e, csm_inc, csm_raan, csm_omega, csm_f])
csm_state_vec           = keplerian_to_state_vec(csm_orbital_elements, mu_moon)

enu_pos             = convert_inertial_state_to_enu(mcmf_state[0:3], launch_site)
q_inertial_to_enu   = get_inertial_to_enu(launch_site)
rot_vec             = quaternion.rotate_vectors(q_inertial_to_enu.conjugate(), np.array([0, 1, 0]))
q_inertial_to_x_up  = q_inertial_to_enu * quaternion.from_rotation_vector(np.radians(90)*rot_vec)
rot_vec             = quaternion.rotate_vectors(q_inertial_to_x_up.conjugate(), np.array([1, 0, 0]))
q_inertial_to_body  = q_inertial_to_x_up * quaternion.from_rotation_vector(np.radians(170)*rot_vec)

omega_body = np.zeros(3)

mass_dry        = 2150.028  # kg
mass            = 4888.18 # kg
J               = np.diag([9230, 4711, 8096])
T_APS           = 15570     # N
Isp_APS         = 311       # s
veh_params      = [mu_moon, J, mass_dry, T_APS, Isp_APS]

att_hold_end            = 2      #sec
vert_rise_radial_rate   = 12.192 #m/s

t_ig                = 5                                      #sec (time of ignition)
R_D                 = 18288 + np.linalg.norm(mci_state[0:3]) #m   (Desired injection radius wrt. launch radius)
Y_D                 = 3148.4                                 #m   (Desired injection out-of-plane distance)
R_D_dot             = 9.81456                                #m/s (Desired injection radial velocity)
Y_D_dot             = 0                                      #m/s (Desired injection cross_range velocity)
Z_D_dot             = 1687.03752                             #m/s (Required injection downrange velocity)

t_go_init           = 370   #sec (Time to go)
t_3                 = 10    #sec (t_go at which time rate coefficients are set to 0)
t_2                 = 2     #sec (t_go at which guidance coefficents are held constant)

mode_switch_states      = [t_ig, att_hold_end, vert_rise_radial_rate]
guid_man_input_params   = [t_ig, R_D, Y_D, R_D_dot, Y_D_dot, Z_D_dot]
guid_auto_input_params  = [veh_params, t_go_init, t_3, t_2]
guidance_params         = [csm_state_vec, mode_switch_states, guid_man_input_params, guid_auto_input_params]

state       = np.hstack((mcmf_state, quaternion.as_float_array(q_inertial_to_body), omega_body, mass))
times       = np.arange(0, 700, .1)
sim_data    = simulate(state, times, veh_params, guidance_params)
sim_data.append(guidance_params)
np.save(save_file, sim_data, allow_pickle=True)